# Preprocessin for Sentiment Analysis
> * [공공데이터포털](https://www.data.go.kr/) : 뉴스빅데이터 분석 정보
> * 수정일 2017.12.26 

In [6]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import re

In [2]:
path1 = './data/news/have_negative_positive/constitution/'
path2 = './data/news/have_negative_positive/household_debt/'
path3 = './data/news/have_negative_positive/olymphic/'
path4 = './data/news/have_negative_positive/'
pathList = [path1, path2, path3, path4]

In [3]:
def Get_CSVfileList(path):
    filelist = glob(path+'*.csv')
    filelist.extend(glob(path+'*.CSV'))
    return filelist

In [65]:
df_data=pd.DataFrame()
x = 0 
for path in pathList:
    fileList = Get_CSVfileList(path)
    for file in fileList:
        df = pd.read_csv(file,encoding='utf-8')
        x += df.shape[0]
        if len(df_data) == 0:
            df_data = df[['감성분석', '긍정 문장', '부정 문장']]
        else:
            df_data = pd.concat([df_data, df[['감성분석', '긍정 문장', '부정 문장']]])
df_data.reset_index(drop=True, inplace=True)        
print (x)
print (df_data.shape)
df_data.head()

176643
(176643, 3)


,감성분석,긍정 문장,부정 문장
0,NEGATIVE 폭압 NEGATIVE 사이비 NEGATIVE 족쇄 NEGATIVE ...,주택조합 승인 자체가 몹시 어려웠던 시절 한국기자협회 주택조합 형식으로 추진된 이 ...,[도청도설] 기자 아파트 단독주택 420여 가구를 짓는 이 사업은 꼭 5년이 걸려 ...
1,NEGATIVE 문제,"서 의원은 여의도 당사에서 기자회견을 열어 “압도적으로 승리할 것이고 국민, 당원이...",김 의원은 여의도 당사 기자회견에서 “대통령이 와서 지난 대선 때 도와준 핵심 간부...
2,NEGATIVE 제외 NEGATIVE 의혹 NEGATIVE 비리 NEGATIVE 포...,박근혜 대통령이 14일 열린 새누리당 전당대회에 참석한 것은 그만큼 아직 여권에서 ...,실제로 1987년 이후 역대 대통령들은 집권 초에는 여당에 막강한 영향력을 행사했지...
3,POSITIVE 최선 POSITIVE 열심히 NEGATIVE 비판 NEGATIVE ...,"“살기 좋은 전북 만들기, 중앙정부와 연결고리 최선” “월드컵에 출전한 우리 국가대...","‘장기 비전이 없다면, 살아남을 수 없다’, ‘플랜B(위기 대응책)없이는 살아남을 ..."
4,POSITIVE 고맙 NEGATIVE 탈락 NEGATIVE 어렵 POSITIVE 구...,"""여러분 너무나 고맙고 감사합니다. 도와주신 여러분께 저의 마음속의 큰 절을 받아주...",공천 탈락 직후 백의종군한 김무성 당시 선대본부장이 당을 떠나며 말없이 남긴 석 줄...


In [351]:
df_words = df_data['감성분석'].copy()
df_positive = df_data['긍정 문장'].copy()
df_negative = df_data['부정 문장'].copy()

* drop list
> 기자 & 이메일 주소만 있는 문장  
> Neutral 제거

## 긍정 문장 추출

### NaN & ' '제거

In [352]:
df_positive.dropna(inplace = True)
df_positive.reset_index(drop=True, inplace = True)
df_positive.drop(df_positive[df_positive.str.match('^ $')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(156189,)


### #*****? 인 값 제거

In [353]:
df_positive.drop(df_positive[df_positive.str.match('^#[a-zA-Z0-9]+\?$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(156116,)


### #*****! 인 값 제거

In [354]:
df_positive.drop(df_positive[df_positive.str.match('^#[a-zA-Z0-9]+\!$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(156089,)


### 이메일만 있는 값 제거

In [355]:
df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')].index,inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(155267,)


### 이메일과 기자명만 있는 값 제거

In [367]:
df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index , inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-. ]*$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[가-힣]{3,3}기자')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[. ][가-힣]{3,3}기자')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[가-힣 ]{3,4}기자$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_. ]+[가-힣 ]{3,4}기자$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+@$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+@([a-zA-Z0-9-]+\.[a-zA-Z0-9-.])*')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('.[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[가-힣+_.=]+?[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.len() < 11].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[(df_positive.str.match('^[가-힣]{3,5}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\| @]*?')) & (df_positive.str.len() < 100)].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

print (df_positive[df_positive==''])

df_positive = df_positive.apply(html.unescape)

df_positive.drop_duplicates(inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)
df_positive = df_positive.apply(html.unescape)
df_positive.drop_duplicates(inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(144438,)


In [350]:
df_positive.to_csv('./data/sentiment_data/positive_sentiment_sentence_from_news.csv', index=False, header= False)

## 부정 문장 추출

### NaN & ' '제거

In [368]:
df_negative.dropna(inplace = True)
df_negative.reset_index(drop=True, inplace = True)
df_negative.drop(df_negative[df_negative.str.match('^ $')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160872,)


### #*****? 인 값 제거

In [369]:
df_negative.drop(df_negative[df_negative.str.match('^#[a-zA-Z0-9]+\?$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160832,)


### #*****! 인 값 제거

In [370]:
df_negative.drop(df_negative[df_negative.str.match('^#[a-zA-Z0-9]+\!$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160832,)


### 이메일만 있는 값 제거

In [371]:
df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')].index,inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160832,)


### 이메일과 기자명만 있는 값 제거

In [372]:
df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index , inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-. ]*$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[가-힣]{3,3}기자')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[. ][가-힣]{3,3}기자')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[가-힣 ]{3,4}기자$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_. ]+[가-힣 ]{3,4}기자$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+@$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+@([a-zA-Z0-9-]+\.[a-zA-Z0-9-.])*')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('.[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[가-힣+_.=]+?[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.len() < 11].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[(df_negative.str.match('^[가-힣]{3,5}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\| @]*?')) & (df_negative.str.len() < 100)].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

print (df_negative[df_negative==''])

df_negative = df_negative.apply(html.unescape)

df_negative.drop_duplicates(inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)
df_negative = df_negative.apply(html.unescape)
df_negative.drop_duplicates(inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160612,)
(160558,)
(160543,)
(160536,)
(160510,)
(160227,)
(160140,)
(160140,)
(159943,)
(159863,)
(159699,)
(159500,)
(158503,)
(158341,)
Series([], Name: 부정 문장, dtype: object)
(151974,)
(151972,)


In [373]:
df_negative.to_csv('./data/sentiment_data/negative_sentiment_sentence_from_news.csv', index=False, header=False)

## 긍정 / 부정 단어 추출

In [374]:
df_words.dropna(inplace = True)
df_words.reset_index(drop = True, inplace = True)
df_words.drop(df_words[df_words.str.match('^ $')].index, inplace = True)
df_words.reset_index(drop = True, inplace = True)
df_words = df_words.apply(html.unescape)
print(df_words.shape)

(167087,)


In [375]:
df_words = df_words[df_words.str.match('NEGATIVE|POSITIVE')]
df_words.reset_index(drop = True, inplace = True)
print (df_words.shape)

(164046,)


In [376]:
negativeWords = df_words.str.findall('(NEGATIVE [가-힣]+)').str.join(' ')
negativeWords.dropna(inplace = True)
negativeWords.reset_index(drop=True, inplace = True)
positiveWords = df_words.str.findall('(POSITIVE [가-힣]+)').str.join(' ')
positiveWords.dropna(inplace = True)
positiveWords.reset_index(drop=True, inplace = True)
print (negativeWords.shape, positiveWords.shape)

(164046,) (164046,)


In [377]:
positiveWords.drop(positiveWords[positiveWords==''].index, inplace = True)
positiveWords.reset_index(drop=True, inplace = True)
negativeWords.drop(negativeWords[negativeWords==''].index, inplace = True)
negativeWords.reset_index(drop=True, inplace = True)
print (negativeWords.shape, positiveWords.shape)
positiveWords = positiveWords.str.findall('([가-힣]+)')
negativeWords = negativeWords.str.findall('([가-힣]+)')

(144425,) (123623,)


In [378]:
positiveWords = list(set(itertools.chain.from_iterable(positiveWords)))
negativeWords = list(set(itertools.chain.from_iterable(negativeWords)))
print (len(positiveWords), len(negativeWords))

5010 7608


In [379]:
positiveWords = pd.Series(positiveWords)
negativeWords = pd.Series(negativeWords)

In [381]:
positiveWords.to_csv('./data/sentiment_data/positive_sentiment_word_from_news.csv', index=False, header=False)
negativeWords.to_csv('./data/sentiment_data/negative_sentiment_word_from_news.csv', index=False, header=False)